In [5]:
# SIFT 특징점 검출
import cv2 as cv

# 영상 읽기
img = cv.imread('./opencv_data/mot_color70.jpg')      # 영상 읽기

# # gray 영상으로 변환
# gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# cv.imshow('sift 전 - gray', gray)

# sift 적용
sift = cv.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)

gray = cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv.imshow('sift', gray)

cv.waitKey()
cv.destroyAllWindows()

In [4]:
# SIFT 특징점 검출
import cv2 as cv

# 영상 읽기
img = cv.imread('./opencv_data/mot_color83.jpg')      # 영상 읽기

# # gray 영상으로 변환
# gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# cv.imshow('sift 전 - gray', gray)

# sift 적용
sift = cv.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)

gray = cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv.imshow('sift', gray)

cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv
import numpy as np
import time

img1=cv.imread('./opencv_data/mot_color70.jpg')[190:350,440:560] # 버스를 크롭하여 모델 영상으로 사용
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
img2=cv.imread('./opencv_data/mot_color83.jpg')              # 장면 영상
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)
print('특징점 개수:',len(kp1),len(kp2)) 

start=time.time()
flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match=flann_matcher.knnMatch(des1,des2,2)

T=0.7       # 임계값
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)
print('매칭에 걸린 시간:',time.time()-start) 

img_match=np.empty((max(img1.shape[0],img2.shape[0]),img1.shape[1]+img2.shape[1],3),dtype=np.uint8)
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('Good Matches', img_match)

k=cv.waitKey()
cv.destroyAllWindows()

특징점 개수: 231 4096
매칭에 걸린 시간: 0.06824326515197754


In [ ]:
import cv2 as cv
import numpy as np

img1=cv.imread('./opencv_data/mot_color70.jpg')[190:350,440:560] # 버스를 크롭하여 모델 영상으로 사용
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
img2=cv.imread('./opencv_data/mot_color83.jpg')              # 장면 영상
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)

flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match=flann_matcher.knnMatch(des1,des2,2)   # 최근접 2개

T=0.7
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)

points1=np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2=np.float32([kp2[gm.trainIdx].pt for gm in good_match])

H,_=cv.findHomography(points1,points2,cv.RANSAC)

h1,w1=img1.shape[0],img1.shape[1]       # 첫 번째 영상의 크기
h2,w2=img2.shape[0],img2.shape[1]       # 두 번째 영상의 크기

box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2)       # 버스 이미지 크기의 box
box2=cv.perspectiveTransform(box1,H)

# 위에서 얻어진 값을 통해 이미지에 그림 그리기
img2=cv.polylines(img2,[np.int32(box2)],True,(0,255,0),8)

img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8)
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
   
cv.imshow('Matches and Homography',img_match)

k=cv.waitKey()
cv.destroyAllWindows()